In [2]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import requests

import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import messagebox 
from tkinter.filedialog import asksaveasfilename

In [27]:
def main_fun(product):
#     prod_df_list = []
#     for product in product_list:
     # Establish connection to the main page and return the data as a BeautifulSoup Object
    base_url = "https://www.accessdata.fda.gov/scripts/cder/ob/"
    url = 'https://www.accessdata.fda.gov/scripts/cder/ob/search_product.cfm' 
    post_params = {'drugname': product, 'discontinued': 'RX,OTC,DISCN','submit':'Search'}
    print(post_params['drugname'])
    response = requests.post(url, data=post_params)

    soup = BeautifulSoup(response.text, 'lxml')

    # fatching Results for Proprietary Name, Active Ingredient or Application Number and approval date etc in myData table
    head = []
    for headers in soup.table.thead.find_all('th'):
        head.append(headers.text)

    myData = pd.DataFrame(columns = head)


    for j in soup.table.tbody.find_all('tr'):

        row_data = j.find_all('td')
        row = [i.text for i in row_data]
        row =[r.replace("\n","") for r in row]
        length = len(myData)
        myData.loc[length] = row

    myData.drop(['Potency_Sort_1', 'TE Code', 'RLD', 'Potency_Sort_2','RS'],axis=1,inplace=True)

    # There may be any product which has no data present in the database so ...
#     return myData

    if len(myData)==0:
        pass
#             messagebox.showerror("error","no data available")

    else:
    # in myData table there are application_numbers linked to other web page , to get those links...
        tr=soup.table.tbody.find_all('tr')
        df4_l=[]
        lis=set()


        #getting link from application number link tag
        
        for apn in tr:
            l=apn.find_all('a',{'id':'id'})
            for i in l:
                lis.add(i['href'].split('#')[0])


        for link1 in lis:

            new_url = base_url+link1
            print(new_url)
            res=requests.get(new_url)
            soup_2 = BeautifulSoup(res.text,"html.parser")
            links=[]
            content = soup_2.find('div',{'class':'ui-accordion ui-widget ui-helper-reset'})
            for l_1 in content.find_all('div'):
                for c in l_1.find_all('a'):
                    links.append(c['href'])

#             apn = soup_2.find('h3',{'class':'search-title'}).text[-6:]
#             application_no ="N"+apn

            header2=[]     
    
            df1 = []
            for link in links:
                new_url2 = base_url+link
                res2 = requests.get(new_url2)
                soup3=BeautifulSoup(res2.text,'html.parser')

                for head1 in soup3.table.thead.find_all('th'):
                    header2.append(head1.text)
                df2 = pd.DataFrame(columns = header2)
                header2.clear()

                for j1 in soup3.table.tbody.find_all('tr'):
                    row_data = j1.find_all('td')
                    row = [i.text for i in row_data]
                    length = len(df2)
                    df2.loc[length] = row
                df1.append(df2)

            df3=pd.concat(df1,ignore_index=True)
            df4_l.append(df3)
        df4=pd.concat(df4_l,ignore_index=True)
        df4.drop(['Drug Substance','Drug Product','Patent Use Code','Delist Requested'],axis=1,inplace=True)
#         return df4
        final_df_list = []
        for prod_no in myData['Product Number']:
            print(prod_no)

            df4_1=df4[df4['Product No']==prod_no]
            mydata_1=myData[myData['Product Number']==prod_no]

            f_df = pd.merge(mydata_1,df4_1,how='cross')
            f_df=f_df.T.drop_duplicates().T
            final_df_list.append(f_df)
        final_df = pd.concat(final_df_list,ignore_index=True)

        return final_df

#             prod_df_list.append(final_df)
            
#     return prod_df_list

main_fun("opana er")


opana er
https://www.accessdata.fda.gov/scripts/cder/ob/results_product.cfm?Appl_Type=N&Appl_No=021610
https://www.accessdata.fda.gov/scripts/cder/ob/results_product.cfm?Appl_Type=N&Appl_No=201655
001
002
003
004
005
006
007
001
002
003
004
005
006
007


,Mkt.Status,Active Ingredient,Proprietary Name,Appl. No.,Product Number,Dosage Form,Route,Strength,Applicant Holder,Approval Date,Patent No,Patent Expiration,Submission Date
0,DISCN,OXYMORPHONE HYDROCHLORIDE,OPANA ER,N021610,001,"TABLET, EXTENDED RELEASE",ORAL,5MG **Federal Register determination that prod...,ENDO PHARMACEUTICALS INC,"Jun 22, 2006",7276250,02/04/2023,
1,DISCN,OXYMORPHONE HYDROCHLORIDE,OPANA ER,N021610,001,"TABLET, EXTENDED RELEASE",ORAL,5MG **Federal Register determination that prod...,ENDO PHARMACEUTICALS INC,"Jun 22, 2006",8309112,02/04/2023,03/18/2021
2,DISCN,OXYMORPHONE HYDROCHLORIDE,OPANA ER,N021610,001,"TABLET, EXTENDED RELEASE",ORAL,5MG **Federal Register determination that prod...,ENDO PHARMACEUTICALS INC,"Jun 22, 2006",8329216,02/04/2023,03/18/2021
3,DISCN,OXYMORPHONE HYDROCHLORIDE,OPANA ER,N021610,001,"TABLET, EXTENDED RELEASE",ORAL,5MG **Federal Register determination that prod...,ENDO PHARMACEUTICALS INC,"Jun 22, 2006",8808737,06/21/2027,03/18/2021
4,DISCN,OXYMORPHONE HYDROCHLORIDE,OPANA ER,N021610,001,"TABLET, EXTENDED RELEASE",ORAL,5MG **Federal Register determination that prod...,ENDO PHARMACEUTICALS INC,"Jun 22, 2006",8871779,11/22/2029,03/18/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,DISCN,OXYMORPHONE HYDROCHLORIDE,OPANA ER,N201655,007,"TABLET, EXTENDED RELEASE",ORAL,40MG,ENDO PHARMACEUTICALS INC,"Dec 9, 2011",8309060,11/20/2023,
388,DISCN,OXYMORPHONE HYDROCHLORIDE,OPANA ER,N201655,007,"TABLET, EXTENDED RELEASE",ORAL,40MG,ENDO PHARMACEUTICALS INC,"Dec 9, 2011",8309122,02/04/2023,
389,DISCN,OXYMORPHONE HYDROCHLORIDE,OPANA ER,N201655,007,"TABLET, EXTENDED RELEASE",ORAL,40MG,ENDO PHARMACEUTICALS INC,"Dec 9, 2011",8329216,02/04/2023,
390,DISCN,OXYMORPHONE HYDROCHLORIDE,OPANA ER,N201655,007,"TABLET, EXTENDED RELEASE",ORAL,40MG,ENDO PHARMACEUTICALS INC,"Dec 9, 2011",8808737,06/21/2027,11/07/2014


In [3]:
win = Tk()
win.state("zoomed")
# win.geometry("1350x1000")
win.resizable(width=False,height= False)
win.config(bg="black")
win.title('fetching data from FDA website')

def download():
    if len(df_final)==1:
        data = [("csv file(*.csv)","*.csv")]
        file = asksaveasfilename(filetypes = data, defaultextension = data)
        with open(file,"w") as f:
            f.write(df_final[0].to_csv())
    else:
        for j in range(1,len(df_final)+1):   
            messagebox.showinfo("success",f"Enter the name of {j}st product:")
            data = [("csv file(*.csv)","*.csv")]
            file = asksaveasfilename(filetypes = data, defaultextension = data)
       # file will have file name provided by user.
       # Now we can use this file name to save file.
            with open(file,"w") as f:
                f.write(df_final[j-1].to_csv())

    messagebox.showinfo("title","files downloaded..")
    main_screen()

def view():
    frame1 = Frame(win,width=1350,height=1000,bg="black")
    frame1.pack()
#     print(df_final[0])
    columns=list(df_final[0].columns)
    scrollbar = Scrollbar(frame1,orient='vertical')
    scrollbar.pack(side='right',fill=Y)

    scrollbar1 = Scrollbar(frame1,orient='horizontal')
    scrollbar1.pack(side='bottom' , fill=X)

    tree = ttk.Treeview(frame1,yscrollcommand=scrollbar.set,xscrollcommand=scrollbar1.set, columns=columns,show='headings',height=30 )
    for i in columns:
        tree.heading(i,text=f"{i}")
        tree.column(i,width=160)

    contacts = df_final[0].to_numpy().tolist()
    for contact in contacts:
        tree.insert('', tk.END, values=contact)

    tree.pack(side=TOP,anchor='w')

    scrollbar.config(command=tree.yview)

    scrollbar1.config(command=tree.xview)
        
        
    
    def download_data():
        frame1.destroy()
        download()
    
    def back():
        frame1.destroy()
        main_screen()


    download_button = Button(frame1,text='Download Data',command=download_data,bg="blue",fg="white",width=23,height=2).pack(side='bottom',anchor='c')
    back_button = Button(frame1,text='Home Page',command=back,bg="blue",fg="white",width=23,height=2).pack(side='bottom',anchor='c')
    



def main_screen():
    frame = Frame(win,width=1350,height=1000,bg="black")
    frame.pack()
    
    title = Label(frame,text="...WELCOME...",font="Helvetica 25 bold",fg="white",bg="black")
    title.place(relx=.38,rely=.1)
    
    
    product_name = Label(frame,text="Enter the Product Name :",font="Helvetica 20 bold",fg="brown",bg="black")
    product_name.place(relx=.2,rely=.4)
    product_name_entry = Entry(frame,font="Helvetica 20 bold",bd=3)
    product_name_entry.place(relx=.5,rely=.4)
    
    txt = Label(frame,text="For Downloading Multiple products,enter commman seprated values :",font="Helvetica 12 bold",fg="white",bg="black")
    txt.place(relx=.5,rely=.35)
    
    
    
    def go_download_page():
        p_name=product_name_entry.get()
        if len(p_name)==0:
            messagebox.showerror("error","Name can not be Empty")
        else:
            p_name=p_name.split(',')
            global df_final
            df_final = main_fun(p_name)
            
            if len(df_final)==0:
                main_screen()
            else:
                frame.destroy()
                download()

    
    def go_to_view_page():
        p_name=product_name_entry.get()
        if len(p_name)==0:
            messagebox.showerror("error","Name can not be Empty")
        else:
            p_name=p_name.split(',')
            global df_final
            df_final = main_fun(p_name)
            print(len(df_final))
            if len(df_final)==0:
                main_screen()
            else:
                frame.destroy()
                view()
            
            
    
    download_button = Button(frame,text='Download Data',command=go_download_page,bg="blue",fg="white",width=23,height=2).place(relx=.3,rely=.6)
    view_button = Button(frame,text='View Data',command=go_to_view_page,bg="blue",fg="white",width=23,height=2).place(relx=.5,rely=.6)

main_screen()
win.mainloop()